In [ ]:
pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 385 kB 30.0 MB/s 
     |████████████████████████████████| 59 kB 1.2 MB/s 
     |████████████████████████████████| 140 kB 85.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [ ]:
pip install pyspellchecker==0.5.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 33.8 MB/s 


In [ ]:
import pandas as pd
import openpyxl
import random
import math
import collections.abc
import copy
import ast
import json
from google.colab import files
import elasticsearch
from elasticsearch import Elasticsearch
import spellchecker
from spellchecker import SpellChecker
import numpy

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
!git clone "<github_json_data_repo_link>"

Cloning into 'IR_Project'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 122 (delta 101), reused 106 (delta 97), pack-reused 0
Receiving objects: 100% (122/122), 15.85 MiB | 24.29 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Checking out files: 100% (107/107), done.


In [ ]:
%cd /content/IR_Project
!git pull

/content/IR_Project
Already up to date.


In [ ]:
calendar_df=pd.read_excel("/content/IR_Project/dimdate-QueryResult.xlsx")

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
calendar_df=calendar_df.drop(['word_date','date','quarter','yearmonth','week','month_and_year','timezone_offset','timezone','timezone_id'],axis=1)

print(calendar_df)

      sasdate  date_key  year  month  day_of_month day_of_week  weekday  \
0       14610     10100  2000    1.0           1.0         Sat      7.0   
1       14611     10200  2000    1.0           2.0         Sun      1.0   
2       14612     10300  2000    1.0           3.0         Mon      2.0   
3       14613     10400  2000    1.0           4.0         Tue      3.0   
4       14614     10500  2000    1.0           5.0         Wed      4.0   
...       ...       ...   ...    ...           ...         ...      ...   
9386    23996     91225  2025    9.0          12.0         Fri      6.0   
9387    23997     91325  2025    9.0          13.0         Sat      7.0   
9388    23998     91425  2025    9.0          14.0         Sun      1.0   
9389    23999     91525  2025    9.0          15.0         Mon      2.0   
9390    24000     91625  2025    9.0          16.0         Tue      3.0   

      holiday  
0     NEWYEAR  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
...

In [ ]:
#contains all the generated calendars
calendar_index_date_key=pd.DataFrame(columns=['user','datekey','sasdate','year','month','day','weekday_num','event'])


In [ ]:
calendar_index_generated_events=pd.DataFrame(columns=['datekey','sasdate','year','month','day','weekday_num','event'])
date_list=pd.DataFrame(columns=['datekey','sasdate','year','month','day','weekday_num'])
for row in calendar_df.iterrows():
    if(isinstance(row[1]['holiday'],str)):
        new_row=[row[1]['date_key'],row[1]['sasdate'],row[1]['year'],row[1]['month'],row[1]['day_of_month'],row[1]['weekday'],str(row[1]['holiday'].lower().split())]
        calendar_index_generated_events.loc[len(calendar_index_generated_events)]=new_row

    date_list.loc[len(date_list)]=[row[1]['date_key'],row[1]['sasdate'],row[1]['year'],row[1]['month'],row[1]['day_of_month'],row[1]['weekday']]
print(calendar_index_generated_events)


    datekey sasdate  year  month   day  weekday_num               event
0     10100   14610  2000    1.0   1.0          7.0         ['newyear']
1     11700   14626  2000    1.0  17.0          2.0             ['mlk']
2     21400   14654  2000    2.0  14.0          2.0      ['valentines']
3     22100   14661  2000    2.0  21.0          2.0    ['uspresidents']
4     42300   14723  2000    4.0  23.0          1.0          ['easter']
..      ...     ...   ...    ...   ...          ...                 ...
469   52625   23887  2025    5.0  26.0          2.0        ['memorial']
470   61525   23907  2025    6.0  15.0          1.0         ['fathers']
471   70125   23923  2025    7.0   1.0          3.0          ['canada']
472   70425   23926  2025    7.0   4.0          6.0  ['usindependence']
473   90125   23985  2025    9.0   1.0          2.0           ['labor']

[474 rows x 7 columns]


In [ ]:
json_generated_events = calendar_index_generated_events.to_json(orient="table")
json_load=json.loads(json_generated_events)
json_dump=json.dumps(json_load,indent=4)
#utf8=json_generated_events.encode('utf-8')

In [ ]:
with open('generated_events.json','w') as event_file:
  event_file.write(json_dump)

files.download('generated_events.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('generated_events_csv.csv','w') as event_file:
  event_file.write(json_dump)

files.download('generated_events_csv.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
calendar_index_generated_events=calendar_index_generated_events.set_index('sasdate')
json_generated_events = calendar_index_generated_events.to_json(orient="table")
json_load=json.loads(json_generated_events)
json_dump=json.dumps(json_load,indent=4)

print(calendar_index_generated_events)
#del json_load['index']
json_dump=json.dumps(json_load,indent=4)
with open('generated_events_sasdate.json','w') as event_file:
  event_file.write(json_dump)

files.download('generated_events_sasdate.json')

        datekey  year  month   day  weekday_num               event
sasdate                                                            
14610     10100  2000    1.0   1.0          7.0         ['newyear']
14626     11700  2000    1.0  17.0          2.0             ['mlk']
14654     21400  2000    2.0  14.0          2.0      ['valentines']
14661     22100  2000    2.0  21.0          2.0    ['uspresidents']
14723     42300  2000    4.0  23.0          1.0          ['easter']
...         ...   ...    ...   ...          ...                 ...
23887     52625  2025    5.0  26.0          2.0        ['memorial']
23907     61525  2025    6.0  15.0          1.0         ['fathers']
23923     70125  2025    7.0   1.0          3.0          ['canada']
23926     70425  2025    7.0   4.0          6.0  ['usindependence']
23985     90125  2025    9.0   1.0          2.0           ['labor']

[474 rows x 6 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(calendar_index_generated_events['event'])

sasdate
14610           ['newyear']
14626               ['mlk']
14654        ['valentines']
14661      ['uspresidents']
14723            ['easter']
                ...        
23887          ['memorial']
23907           ['fathers']
23923            ['canada']
23926    ['usindependence']
23985             ['labor']
Name: event, Length: 474, dtype: object


In [ ]:
#created random eventd on 100K calendars in batches of 10K
date=0
events = ['meeting','Meeting','meet','Birthday','birthday','bday','anivarsary','aniversary','Annivarsary','anniversary',
         'wedding anniversary','meetings','party','event','birthday party']
people=['tom','tim','jim','tina','shivani bhalerao','neel','saloni','mummy','pappa','mom','dad','mom and dad','uncle','Joe','Joey','rhea',
       'john','johny']
festivals=['Ganesh Chaturthi','Diwali','Divali','holi','ShivaJayanti','pola','Gudhi Padwa','Thanks giving','halloween','Haloween']
calendar_index_date_key=pd.DataFrame(columns=['user','datekey','sasdate','year','month','day','weekday_num','event'])
len_calendar_index_date_key=0

for i in range(90001,100001):

    user='user_'+str(i)
    for person_and_event in range(0,10):
        random_event=events[random.randrange(len(events))]
        random_subject=people[random.randrange(len(people))]
        event_people_combination=[random_event+' '+random_subject,random_subject+' '+random_event]
        event_people=random.choice(event_people_combination)
        
        random_num=random.randint(0,len(date_list)-1)
        random_date=date_list.iloc[random_num]
        random_datekey=random_date['datekey']
        
        random_sasdate=random_date['sasdate'].astype(str).astype(float).astype(int)
        
        prev_row=calendar_index_date_key[(calendar_index_date_key['user']==user) & (calendar_index_date_key['sasdate'].astype(str).astype(float).astype(int)==random_sasdate)]
        if(len(prev_row)>0):
            prev_event_str=prev_row['event'].tolist()
            prev_event_str=prev_event_str[0]
            prev_event=ast.literal_eval(prev_event_str)
            prev_event.append(event_people)
            calendar_index_date_key.loc[((calendar_index_date_key['user']==user) & (calendar_index_date_key['sasdate'].astype(str).astype(float).astype(int)==random_sasdate)),'event']=str(prev_event)

        else:
            prev_event=list([])
            prev_event.append(event_people)
            new_event_row=[user,random_date['datekey'],random_date['sasdate'],random_date['year'],random_date['month'],random_date['day'],random_date['weekday_num'],str(prev_event)]
            #calendar_index_date_key.loc[((calendar_index_date_key['user']==user) & (calendar_index_date_key['sasdate'].astype(str).astype(int)==random_sasdate)),'event']=str(prev_event)
            calendar_index_date_key.loc[len_calendar_index_date_key]=new_event_row
            len_calendar_index_date_key=len_calendar_index_date_key+1
            #print(calendar_index_date_key[calendar_index_date_key['sasdate']==random_sasdate])
        #print(random_date,' ',random_event,' ',random_subject,' ',user,' ',tuple([user,random_date['datekey']]))
        
    for festival in range(0,2):
        random_festival=festivals[random.randrange(len(festivals))]
        
        random_num=random.randint(0,len(date_list)-1)
        random_date=date_list.iloc[random_num]
        random_datekey=random_date['datekey']
        
        random_sasdate=random_date['sasdate'].astype(str).astype(float).astype(int)
        
        prev_row=calendar_index_date_key[(calendar_index_date_key['user']==user) & (calendar_index_date_key['sasdate'].astype(str).astype(float).astype(int)==random_sasdate)]

        #print(user,' ',random_sasdate)

        if(len(prev_row)>0):
            prev_event_str=prev_row['event'].tolist()
            #print('before',prev_event_str)
            prev_event_str=prev_event_str[0]
            prev_event=ast.literal_eval(prev_event_str)
            prev_event.append(random_festival)
            calendar_index_date_key.loc[((calendar_index_date_key['user']==user) & (calendar_index_date_key['sasdate'].astype(str).astype(float).astype(int)==random_sasdate)),'event']=str(prev_event)
    
        else:
            prev_event=list([])
            prev_event.append(random_festival)
            new_event_row=[user,random_date['datekey'],random_date['sasdate'],random_date['year'],random_date['month'],random_date['day'],random_date['weekday_num'],str(prev_event)]
            #calendar_index_date_key.loc[((calendar_index_date_key['user']==user) & (calendar_index_date_key['sasdate'].astype(str).astype(int)==random_sasdate)),'event']=str(prev_event)
            calendar_index_date_key.loc[len_calendar_index_date_key]=new_event_row
            len_calendar_index_date_key=len_calendar_index_date_key+1
            
    if(i%1000==0):
      calendar_index_date_key=calendar_index_date_key.set_index(['user','sasdate'])
      json_user_event = calendar_index_date_key.to_json(orient="table")
      json_load=json.loads(json_user_event)
      json_dump=json.dumps(json_load,indent=4)
      file_name='user_event_'+str(i)+'.json'
      with open(file_name,'w') as event_file:
        event_file.write(json_dump)
        files.download(file_name)

      calendar_index_date_key=pd.DataFrame(columns=['user','datekey','sasdate','year','month','day','weekday_num','event'])
      len_calendar_index_date_key=0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
prev_row=calendar_index_date_key[((calendar_index_date_key['user']=='user_0') & (calendar_index_date_key['datekey']==111017))]
prev_event_str=prev_row['event'].tolist()
prev_event_str=prev_event_str[0]
prev_event=ast.literal_eval(prev_event_str)
prev_event.append('abc')

print(prev_event)
calendar_index_date_key.loc[((calendar_index_date_key['user']=='user_0') & (calendar_index_date_key['datekey']==10100)),'event']=str(prev_event)
#calendar_index_date_key.loc[((calendar_index_date_key['user']=='user_0') & (calendar_index_date_key['datekey']==10100)),'event']

In [ ]:
print(calendar_index_date_key[calendar_index_date_key['sasdate'].astype(str).astype(float).astype(int)==23944])

           user  datekey  sasdate    year  month   day  weekday_num  \
1013    user_84  72225.0  23944.0  2025.0    7.0  22.0          3.0   
10043  user_837  72225.0  23944.0  2025.0    7.0  22.0          3.0   
11993  user_999  72225.0  23944.0  2025.0    7.0  22.0          3.0   

                  event  
1013   ['meetings tim']  
10043   ['tim Meeting']  
11993      ['Haloween']  


In [ ]:
json_file=open('/content/IR_Project/JSON/user_date_indexed/user_event_1000.json')
dict_json=json.load(json_file)
#test_json=pd.read_json('user_event_1000.json',lines=True,orient='Table')

In [ ]:
print(dict_json['data'][0])
read_json_df=pd.DataFrame(dict_json['data'])


{'user': 'user_1', 'sasdate': 22497.0, 'datekey': 80521.0, 'year': 2021.0, 'month': 8.0, 'day': 5.0, 'weekday_num': 5.0, 'event': "['tim event']"}


In [ ]:

for i in range(2000,100001,1000):
  file_name='/content/IR_Project/JSON/user_date_indexed/user_event_'+str(i)+'.json'
  json_file=open(file_name)
  dict_json=json.load(json_file)
  #print(i)
  read_json_df=read_json_df.append(dict_json['data'],ignore_index=True)


In [ ]:
read_json_df=read_json_df.set_index(['user','sasdate'])

In [ ]:
#takes a lotttttt of time: thus now we will start using only 1000 users for now
N=len(read_json_df)
for i in range(0,1000):
  #print(read_json_df.loc[read_json_df['user']==row[0][0]])
  event_str=read_json_df.loc[(read_json_df.index[i][0],read_json_df.index[i][1]),'event']
  formatted_event=""
  for character in event_str:
      if character.isalnum() or character==' ' or character==',':
        formatted_event=formatted_event+character.lower()
  
  formatteed_event=formatted_event.split(',')
  read_json_df.loc[(read_json_df.index[i][0],read_json_df.index[i][1]),'formatted_event']=formatted_event
  #print(i)
  
  

In [ ]:
read_json_df

datekey    year  month   day  weekday_num  \
user        sasdate                                               
user_1      22497.0   80521.0  2021.0    8.0   5.0          5.0   
            16109.0   20804.0  2004.0    2.0   8.0          1.0   
            17968.0   31209.0  2009.0    3.0  12.0          5.0   
            17692.0   60908.0  2008.0    6.0   9.0          2.0   
            22805.0   60922.0  2022.0    6.0   9.0          5.0   
...                       ...     ...    ...   ...          ...   
user_100000 18849.0   81011.0  2011.0    8.0  10.0          4.0   
            16576.0   52005.0  2005.0    5.0  20.0          6.0   
            21401.0   80518.0  2018.0    8.0   5.0          1.0   
            19277.0  101112.0  2012.0   10.0  11.0          5.0   
            15852.0   52703.0  2003.0    5.0  27.0          3.0   

                                             event  
user        sasdate                                 
user_1      22497.0                  ['tim event']  
            16109.0        ['Joey birthday party']  
            17968.0              ['rhea Birthday']  
            17692.0           ['meet mom and dad']  
            22805.0              ['johny Meeting']  
...                                            ...  
user_100000 18849.0              ['johny meeting']  
            16576.0  ['wedding anniversary pappa']  
            21401.0               ['meeting john']  
            19277.0                       ['pola']  
            15852.0               ['ShivaJayanti']  

[1199300 rows x 6 columns]

In [ ]:
read_json_df[read_json_df['formatted_event'].isna()]

datekey    year  month   day  weekday_num  \
user        sasdate                                               
user_253    16620.0   70305.0  2005.0    7.0   3.0          1.0   
            23143.0   51323.0  2023.0    5.0  13.0          7.0   
            19535.0   62613.0  2013.0    6.0  26.0          4.0   
            15487.0   52702.0  2002.0    5.0  27.0          2.0   
            22899.0   91122.0  2022.0    9.0  11.0          1.0   
...                       ...     ...    ...   ...          ...   
user_100000 18849.0   81011.0  2011.0    8.0  10.0          4.0   
            16576.0   52005.0  2005.0    5.0  20.0          6.0   
            21401.0   80518.0  2018.0    8.0   5.0          1.0   
            19277.0  101112.0  2012.0   10.0  11.0          5.0   
            15852.0   52703.0  2003.0    5.0  27.0          3.0   

                                             event formatted_event  
user        sasdate                                                 
user_253    16620.0                ['Meeting jim']             NaN  
            23143.0          ['aniversary saloni']             NaN  
            19535.0   ['wedding anniversary Joey']             NaN  
            15487.0                 ['pappa meet']             NaN  
            22899.0                   ['Haloween']             NaN  
...                                            ...             ...  
user_100000 18849.0              ['johny meeting']             NaN  
            16576.0  ['wedding anniversary pappa']             NaN  
            21401.0               ['meeting john']             NaN  
            19277.0                       ['pola']             NaN  
            15852.0               ['ShivaJayanti']             NaN  

[1196273 rows x 7 columns]

Proceed to Project V2.ipynb file
